# AI with Buzzer Alerts

## Interactive AI Systems

In the previous lesson, you learned to read AI detection data. Now you'll create an interactive AI system that responds to detections by controlling physical hardware.

## What AI + Hardware Integration Provides

**Detection Data**: The AI identifies objects with confidence scores

**Decision Logic**: Your code decides what to do with the detection

**Hardware Response**: Physical outputs (buzzers, LEDs, motors) respond to AI decisions

**Real-Time Interaction**: The system actively responds to what it sees

## Understanding GPIO

GPIO (General Purpose Input/Output) pins allow the Raspberry Pi to control external hardware:

**Digital Output**: Send HIGH (3.3V) or LOW (0V) signals to control devices like LEDs, buzzers, relays

**Digital Input**: Read HIGH or LOW signals from sensors, buttons, switches

**PWM (Pulse Width Modulation)**: Control brightness, speed, or tone by rapidly switching between HIGH and LOW

## Hardware Setup Required

For this lesson you need:
- **Buzzer** connected to **GPIO Pin 17**
- **Ground connection** from buzzer to any Pi ground pin

**Wiring:**
- Buzzer positive (+) → GPIO Pin 17 (Physical pin 11)  
- Buzzer negative (-) → Ground (Physical pin 14 or any GND pin)

If you don't have a buzzer, you can substitute an LED to see visual alerts instead of audio alerts."

## Test Your Hardware Setup

Let's test the buzzer connection before building the AI system:

In [ ]:
import RPi.GPIO as GPIO
import time

# GPIO Setup for buzzer control
BUZZER_PIN = 17
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(BUZZER_PIN, GPIO.OUT)

print("Testing buzzer connection...")

try:
    # Test buzzer with 3 short beeps
    for i in range(3):
        print(f"Beep {i+1}")
        GPIO.output(BUZZER_PIN, GPIO.HIGH)  # Turn buzzer ON
        time.sleep(0.2)
        GPIO.output(BUZZER_PIN, GPIO.LOW)   # Turn buzzer OFF
        time.sleep(0.3)
    
    print("Buzzer test complete!")
    print("Did you hear the beeps? If yes, you're ready for the AI integration.")
    
except Exception as e:
    print(f"Buzzer test failed: {e}")
    print("Check your wiring and try again")

finally:
    GPIO.cleanup()

## How GPIO Control Works

GPIO control is simple but powerful:

**GPIO.setup(pin, GPIO.OUT)**: Configure a pin as an output

**GPIO.output(pin, GPIO.HIGH)**: Send 3.3V to the pin (turn device ON)

**GPIO.output(pin, GPIO.LOW)**: Send 0V to the pin (turn device OFF)

**GPIO.cleanup()**: Reset all GPIO settings (important for safety)

## The AI + Hardware Pipeline

```
Camera → AI Detection → Decision Logic → GPIO Output → Physical Response
```

1. **Camera** captures video frame
2. **AI Detection** identifies objects with confidence scores  
3. **Decision Logic** evaluates if we should respond (threshold check)
4. **GPIO Output** sends HIGH/LOW signal to buzzer pin
5. **Physical Response** buzzer sounds, alerting humans

This is the foundation of smart doorbells, security cameras, and IoT devices.

## Interactive AI with Configurable Object Detection

This system will detect any object from the COCO dataset and trigger buzzer alerts:

In [ ]:
import sys
import time
import threading
import RPi.GPIO as GPIO
import cv2
import numpy as np
from IPython.display import display, clear_output, Image
import ipywidgets as widgets

# Add picamera2 to path
sys.path.append('/home/pi/picamera2')

from picamera2 import MappedArray, Picamera2
from picamera2.devices import IMX500
from picamera2.devices.imx500 import NetworkIntrinsics

# GPIO Setup
BUZZER_PIN = 17
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(BUZZER_PIN, GPIO.OUT)
GPIO.output(BUZZER_PIN, GPIO.LOW)

# Stop button for camera stream
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop AI Detection',
    button_style='danger',
    icon='square'
)

# HARDCODED VALUES - Experiment by changing these values
TARGET_OBJECT = 'person'  # Change this to experiment with different objects: 'cup', 'apple', 'cell phone', etc.
CONFIDENCE_THRESHOLD = 0.7  # Change this to experiment with different sensitivity levels (0.5 to 0.95)

# Data output area
data_output = widgets.Output()

display(widgets.VBox([
    stopButton,
    data_output
]))

# Global variables for detection system
current_detections = []
total_detection_count = 0
alert_count = 0
last_alert_time = 0
last_results = []  # Store detection results globally

# COCO labels for the model
def get_labels():
    """Get COCO dataset labels"""
    return [
        "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
        "traffic light", "fire hydrant", "", "stop sign", "parking meter", "bench", "bird", "cat",
        "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "", "backpack",
        "umbrella", "", "", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
        "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
        "tennis racket", "bottle", "", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
        "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza",
        "donut", "cake", "chair", "couch", "potted plant", "bed", "", "dining table", "", "",
        "toilet", "", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave",
        "oven", "toaster", "sink", "refrigerator", "", "book", "clock", "vase", "scissors",
        "teddy bear", "hair drier", "toothbrush"
    ]

class Detection:
    def __init__(self, coords, category, conf, metadata, imx500, picam2):
        """Create a Detection object, recording the bounding box, category and confidence."""
        self.category = category
        self.conf = conf
        self.box = imx500.convert_inference_coords(coords, metadata, picam2)

def parse_detections(metadata: dict, imx500, intrinsics, picam2):
    """Parse the output tensor into detected objects, scaled to the ISP output."""
    global last_results, current_detections, total_detection_count
    
    bbox_normalization = intrinsics.bbox_normalization
    bbox_order = intrinsics.bbox_order
    threshold = 0.55  # Base detection threshold
    
    np_outputs = imx500.get_outputs(metadata, add_batch=True)
    input_w, input_h = imx500.get_input_size()
    
    if np_outputs is None:
        return last_results
    
    # Parse detection outputs
    boxes, scores, classes = np_outputs[0][0], np_outputs[1][0], np_outputs[2][0]
    
    if bbox_normalization:
        boxes = boxes / input_h
    
    if bbox_order == "xy":
        boxes = boxes[:, [1, 0, 3, 2]]
    
    boxes = np.array_split(boxes, 4, axis=1)
    boxes = zip(*boxes)
    
    # Create detection objects
    detections = [
        Detection(box, category, score, metadata, imx500, picam2)
        for box, score, category in zip(boxes, scores, classes)
        if score > threshold
    ]
    
    last_results = detections
    return detections

def draw_detections(request, stream="main"):
    """Draw the detections for this request onto the ISP output."""
    global last_results
    detections = last_results
    if not detections:
        return
        
    labels = get_labels()
    with MappedArray(request, stream) as m:
        for detection in detections:
            x, y, w, h = detection.box
            label = f"{labels[int(detection.category)]} ({detection.conf:.2f})"
            
            # Calculate text size and position
            (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            text_x = x + 5
            text_y = y + 15
            
            # Create a copy of the array to draw the background with opacity
            overlay = m.array.copy()
            
            # Draw the background rectangle on the overlay
            cv2.rectangle(overlay,
                          (text_x, text_y - text_height),
                          (text_x + text_width, text_y + baseline),
                          (255, 255, 255),  # Background color (white)
                          cv2.FILLED)
            
            alpha = 0.30
            cv2.addWeighted(overlay, alpha, m.array, 1 - alpha, 0, m.array)
            
            # Draw text on top of the background
            cv2.putText(m.array, label, (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            
            # Draw detection box
            cv2.rectangle(m.array, (x, y), (x + w, y + h), (0, 255, 0, 0), thickness=2)

def trigger_buzzer_alert(object_name, confidence):
    """Trigger buzzer when target object is detected"""
    global alert_count, last_alert_time
    
    current_time = time.time()
    
    # Cooldown period to prevent spam alerts (2 seconds)
    if current_time - last_alert_time < 2.0:
        return
    
    try:
        # Buzzer alert pattern
        GPIO.output(BUZZER_PIN, GPIO.HIGH)
        time.sleep(0.15)  # Beep duration
        GPIO.output(BUZZER_PIN, GPIO.LOW)
        
        alert_count += 1
        last_alert_time = current_time
        
        # Print alert to console
        print(f"🔊 ALERT #{alert_count}: {object_name.upper()} detected with {confidence:.2f} confidence")
        
    except Exception as e:
        print(f"Buzzer alert error: {e}")

def capture_detection_data():
    """Capture and display detection data"""
    global current_detections, total_detection_count, last_results
    
    def update_detection_display():
        """Update our custom detection display"""
        target_object = TARGET_OBJECT  # Use hardcoded value
        threshold = CONFIDENCE_THRESHOLD  # Use hardcoded value
        
        with data_output:
            clear_output(wait=True)
            print("AI + BUZZER ALERT SYSTEM")
            print("=" * 30)
            print(f"🎯 Target Object: {target_object.upper()}")
            print(f"🎚️ Alert Threshold: {threshold:.1%}")
            print(f"📊 Total Detections: {total_detection_count}")
            print(f"🔊 Alerts Triggered: {alert_count}")
            print()
            
            if last_results:
                print("CURRENT DETECTIONS:")
                print("-" * 20)
                
                labels = get_labels()
                for i, detection in enumerate(last_results, 1):
                    x, y, w, h = detection.box
                    conf_pct = detection.conf * 100
                    object_name = labels[int(detection.category)]
                    
                    # Confidence indicator
                    if detection.conf >= 0.8:
                        indicator = "🟢 HIGH"
                    elif detection.conf >= 0.6:
                        indicator = "🟡 MEDIUM"
                    else:
                        indicator = "🟠 LOW"
                    
                    # Highlight target object
                    name_display = f"⭐ {object_name.upper()}" if object_name == target_object else object_name.upper()
                    
                    print(f"Detection {i}: {name_display}")
                    print(f"   Confidence: {conf_pct:.1f}% ({indicator})")
                    print(f"   Location: ({x}, {y})")
                    print(f"   Size: {w} x {h} pixels")
                    print()
            else:
                print("Waiting for detections...")
            
            print("ALERT SYSTEM STATUS:")
            print("-" * 20)
            print(f"🎯 Watching for: {target_object}")
            print(f"🎚️ Confidence needed: {threshold:.1%}")
    
    # Monitor detection changes and update display
    last_detection_count = 0
    while not stopButton.value:
        try:
            if last_results:
                target_object = TARGET_OBJECT  # Use hardcoded value
                threshold = CONFIDENCE_THRESHOLD  # Use hardcoded value
                labels = get_labels()
                
                # Check for new detections of target object
                for detection in last_results:
                    object_name = labels[int(detection.category)]
                    
                    if object_name == target_object and detection.conf >= threshold:
                        # Target object detected with sufficient confidence!
                        trigger_buzzer_alert(object_name, detection.conf)
                
                # Update display if detection count changed
                if len(last_results) != last_detection_count:
                    total_detection_count += len(last_results) - last_detection_count
                    last_detection_count = len(last_results)
                    update_detection_display()
            
            time.sleep(0.5)  # Check for updates every 500ms
        except Exception as e:
            time.sleep(0.5)
            continue

def run_detection_demo():
    """Run the IMX500 detection demo with buzzer integration"""
    global last_results
    
    try:
        # Initialize IMX500 with the object detection model
        model_path = "/usr/share/imx500-models/imx500_network_ssd_mobilenetv2_fpnlite_320x320_pp.rpk"
        imx500 = IMX500(model_path)
        intrinsics = imx500.network_intrinsics
        
        if not intrinsics:
            intrinsics = NetworkIntrinsics()
            intrinsics.task = "object detection"
        
        # Load labels
        intrinsics.labels = get_labels()
        intrinsics.update_with_defaults()
        
        # Initialize camera
        picam2 = Picamera2(imx500.camera_num)
        config = picam2.create_preview_configuration(
            controls={"FrameRate": intrinsics.inference_rate}, 
            buffer_count=12
        )
        
        # Show progress bar for network loading
        imx500.show_network_fw_progress_bar()
        
        # Start camera
        picam2.start(config, show_preview=False)
        
        # Set up aspect ratio if needed
        if intrinsics.preserve_aspect_ratio:
            imx500.set_auto_aspect_ratio()
        
        # Set up drawing callback
        picam2.pre_callback = draw_detections
        
        # Create display handle for video
        display_handle = display(None, display_id=True)
        
        # Main loop
        while not stopButton.value:
            try:
                # Parse detections from metadata
                last_results = parse_detections(picam2.capture_metadata(), imx500, intrinsics, picam2)
                
                # Capture frame for display
                request = picam2.capture_request()
                img = request.make_array("main")
                _, jpeg = cv2.imencode('.jpg', img)
                display_handle.update(Image(data=jpeg.tobytes()))
                request.release()
                
                time.sleep(0.1)
            except Exception as e:
                time.sleep(0.1)
                continue
        
        picam2.stop()
        
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure IMX500 camera is connected and model files exist.")
    finally:
        GPIO.output(BUZZER_PIN, GPIO.LOW)  # Ensure buzzer is off
        GPIO.cleanup()

# Start the detection demo in background
detection_thread = threading.Thread(target=run_detection_demo, daemon=True)
detection_thread.start()

# Start our data capture monitoring
data_thread = threading.Thread(target=capture_detection_data, daemon=True)
data_thread.start()

## Understanding Alert Systems

### Threshold Selection

The confidence threshold determines when alerts are triggered:

**High Threshold (85-95%)**:
- Very reliable alerts
- Few false positives  
- May miss some valid detections
- Best for: Security systems, critical alerts

**Medium Threshold (70-84%)**:
- Balanced accuracy and sensitivity
- Occasional false positives
- Best for: General monitoring, smart home

**Low Threshold (50-69%)**:
- High sensitivity, catches most detections
- More false alerts
- Best for: Research, when missing detections is worse than false alerts

### Cooldown Periods

The 2-second cooldown prevents alert spam:
- Prevents multiple buzzer sounds for the same detection
- Reduces annoying repetitive alerts
- Allows time for object to move out of frame
- Can be adjusted based on use case

## Experiment: Test Different Objects

Try changing the target object and testing with different items:

# Common objects to test with your AI system

test_objects = {
    'person': 'Stand in front of camera',
    'cup': 'Hold up a coffee cup or mug', 
    'cell phone': 'Hold up your phone',
    'book': 'Hold up any book or notebook',
    'apple': 'Hold up an apple or similar fruit',
    'mouse': 'Computer mouse works well',
    'keyboard': 'Computer keyboard',
    'bottle': 'Water bottle or similar',
    'chair': 'Point camera at a chair',
    'laptop': 'Open laptop in view'
}

print("OBJECT DETECTION TEST GUIDE")
print("=" * 30)
print("Try these common objects with your AI alert system:")
print()

for obj, instruction in test_objects.items():
    print(f"📦 {obj.upper()}")
    print(f"   Test: {instruction}")
    print()

print("TESTING PROCEDURE:")
print("1. Select object from dropdown")
print("2. Set confidence threshold (try 70% first)")
print("3. Follow the instruction above")
print("4. Listen for buzzer alert")
print("5. Try adjusting threshold if needed")
print()
print("EXPECTED RESULTS:")
print("• Higher threshold = fewer alerts (more reliable)")
print("• Lower threshold = more alerts (more sensitive)")
print("• Some objects detect better than others")
print("• Lighting and angle affect detection quality")

## Real-World Applications

This AI + GPIO pattern is used in many commercial systems:

**Smart Doorbells**: Detect people and send alerts to phone apps
- Target: 'person' with high confidence threshold
- Response: Push notification + recorded video

**Security Cameras**: Monitor for intruders or specific activities  
- Target: 'person' in restricted areas
- Response: Alarm sound + alert security team

**Smart Retail**: Track customer interactions with products
- Target: Multiple objects (products)
- Response: Update inventory systems

**Wildlife Monitoring**: Camera traps for conservation
- Target: Specific animal species
- Response: Log sighting data + researcher alerts

**Manufacturing Quality Control**: Detect defects on assembly lines
- Target: Product anomalies
- Response: Stop production line + alert operators

### Key Design Principles

1. **Choose appropriate thresholds** based on consequences of false positives/negatives
2. **Implement cooldown periods** to prevent alert spam  
3. **Log all activity** for analysis and improvement
4. **Provide user controls** for threshold adjustment
5. **Test thoroughly** with real-world conditions

The system you just built follows these same professional patterns!

## Advanced: Multiple Alert Patterns

Want to create different buzzer patterns for different objects? Here's how:

In [ ]:
import RPi.GPIO as GPIO
import time

def create_alert_pattern(object_type):
    """Create different buzzer patterns for different objects"""
    
    # Define patterns: (beep_duration, pause_duration)
    patterns = {
        'person': [(0.2, 0.3)],  # Single long beep
        'cup': [(0.1, 0.1), (0.1, 0.1)],  # Two quick beeps  
        'apple': [(0.1, 0.1), (0.1, 0.1), (0.1, 0.3)],  # Three quick beeps
        'cell phone': [(0.05, 0.05), (0.05, 0.05), (0.05, 0.05), (0.05, 0.3)],  # Four very quick beeps
        'mouse': [(0.3, 0.2)],  # One longer beep
        'book': [(0.1, 0.2), (0.2, 0.3)]  # Short then long beep
    }
    
    # Get pattern or use default
    pattern = patterns.get(object_type, [(0.15, 0.2)])  # Default: medium beep
    
    return pattern

def play_alert_pattern(object_type):
    """Play the alert pattern for a specific object"""
    pattern = create_alert_pattern(object_type)
    
    BUZZER_PIN = 17
    GPIO.setmode(GPIO.BCM)
    GPIO.setwarnings(False) 
    GPIO.setup(BUZZER_PIN, GPIO.OUT)
    
    try:
        for beep_time, pause_time in pattern:
            GPIO.output(BUZZER_PIN, GPIO.HIGH)
            time.sleep(beep_time)
            GPIO.output(BUZZER_PIN, GPIO.LOW) 
            time.sleep(pause_time)
    except:
        pass
    finally:
        GPIO.cleanup()

# Demonstrate the different patterns
print("ALERT PATTERN DEMONSTRATIONS")
print("=" * 30)

pattern_descriptions = {
    'person': 'Single long beep',
    'cup': 'Two quick beeps',
    'apple': 'Three quick beeps', 
    'cell phone': 'Four rapid beeps',
    'mouse': 'One longer beep',
    'book': 'Short then long beep'
}

for obj, desc in pattern_descriptions.items():
    print(f"{obj.upper()}: {desc}")

print("\nTo test a pattern manually, run:")
print("play_alert_pattern('person')  # or any other object")
print("\nTo integrate this into your AI system:")
print("Replace the simple buzzer code with play_alert_pattern(object_name)")

# Example integration code
print("\nEXAMPLE INTEGRATION:")
print("=" * 20)
print("""
def trigger_buzzer_alert(object_name, confidence):
    global alert_count, last_alert_time
    
    current_time = time.time()
    if current_time - last_alert_time < 2.0:
        return
        
    play_alert_pattern(object_name)  # Use pattern instead of simple beep
    alert_count += 1
    last_alert_time = current_time
    
    print(f"🔊 ALERT #{alert_count}: {object_name.upper()} detected")
""")

## Summary: AI + Hardware Integration

Congratulations! You've built a complete interactive AI system that bridges the digital and physical worlds.

### What You've Accomplished

✅ **GPIO Control**: Learned to control hardware with Raspberry Pi pins

✅ **AI Integration**: Connected AI detection data to physical responses

✅ **Configurable System**: Built controls for different objects and thresholds

✅ **Real-Time Response**: Created a system that responds instantly to detections

✅ **Professional Architecture**: Used patterns found in commercial AI systems

### Key Concepts Learned

**GPIO Basics**: Digital output control with HIGH/LOW signals

**Alert Thresholds**: Balancing sensitivity vs reliability in AI systems

**Cooldown Logic**: Preventing spam alerts in real-time systems

**Object-Specific Responses**: Different actions for different detections

**System Integration**: Combining multiple components into a working system

### Real-World Applications

The patterns you've learned are used in:
- Smart home security systems
- Industrial automation
- IoT devices and sensors
- Autonomous vehicle safety systems
- Medical monitoring equipment

### Next Steps

Ready for more advanced interactive AI? Try:
- **Multiple Output Devices**: Control LEDs, motors, servos
- **Input Sensors**: Combine camera AI with temperature, motion, sound sensors  
- **Network Integration**: Send alerts to phones, web dashboards, cloud services
- **Machine Learning**: Train custom models for your specific use cases

You now understand the fundamentals of interactive AI systems - congratulations on becoming an AI + hardware integration developer!

### 🧠 Key Concepts Mastered

1. **Detection → Decision → Action Pipeline**
2. **GPIO Hardware Control**
3. **Alert Threshold Optimization**
4. **Real-Time System Design**
5. **User Interface Integration**


### 🎯 Skills You Can Now Build

- **Smart Doorbells**: Detect specific people and alert accordingly
- **Security Systems**: Monitor areas and respond to intrusions
- **IoT Devices**: Create smart sensors that react to their environment
- **Automation Systems**: Trigger actions based on visual input
- **Educational Tools**: Build interactive learning systems